In [2]:
from nilmlab.sql_manager import ResultLogger
import os 
from itertools import product
import pandas as pd

result_logger = ResultLogger(db_name='nilm')
result_logger.create_db()

datasource_name = "ukdale" 
RESULTS_DIR = "/l/users/roberto.guillen/nilm/results/"

In [ ]:
# If result file exists 
ta = []
for i, (components, ts_length, num_rep_vec,classifier_type) in enumerate(product([2**p for p in range(2, 9)], ["10Min","Hour","Day"], [1,2,4,8],[0,1,2,3])):
    exp_name = "SQLday_dataset_%s_components_%d_tsLength_%s_numRepVec_%d_classifier_type_%d"%(datasource_name, components, ts_length, num_rep_vec,classifier_type) 
    # ta.append(i)
    # continue
    results_file_dir = RESULTS_DIR + datasource_name + "/" 
    results_file_dir = os.path.join(results_file_dir, f'{exp_name}.csv')
    if os.path.exists(results_file_dir): # if exists create db entry
        result_logger.insert_result(exp_name)
        df = pd.read_csv(results_file_dir)
        print("storing",exp_name)
        result_logger.update_result(key=exp_name,value=df)
# print(len(ta))

In [ ]:
result_logger.get_results()

In [58]:
import pickle
import pandas as pd

db_results = []

for (name, results) in result_logger.get_results():

    if results is not None:        

        df_results = pickle.loads(results)

        df_results.insert(0, 'exp_name', name)

        db_results.append(df_results)

complete_results = pd.concat(db_results, axis=0)
complete_results["report"] = complete_results["report"].replace({'\'': '"'}, regex=True)
complete_results["report"] = complete_results["report"].apply(json.loads)

# Remove duplicates

complete_results = complete_results.drop_duplicates(subset=['exp_name'], keep='last')

complete_results.to_csv("complete_results.csv")

# print(db_results[1]['train_end_date'])

# df_results = pd.DataFrame.from_dict(db_results)

# db_results[0].insert(0, 'test', 'test')

# df = pd.DataFrame(db_results, columns=['exp_name', 'results'])

# df.head()

# for row in  result_logger.get_results():
    
#     print(pickle.load(row[1]))

In [59]:
sample_report = complete_results['report'].iloc[0]

apliances_columns = []

for key in sample_report.keys():

    for metric in sample_report[key]:
        apliances_columns.append(key + "_" + metric)

# Extract a list of all the metrics

report_as_list = complete_results['report'].to_list()

apliances_kpis = []

for report in report_as_list:

    row_metrics = []

    for key in report.keys():

        for metric in report[key]:
            row_metrics.append(report[key][metric])

    apliances_kpis.append(row_metrics)

In [60]:
complete_results = complete_results.reindex(columns=complete_results.columns.tolist() + apliances_columns) 



complete_results[apliances_columns] = apliances_kpis

In [61]:
apliances_columns

['microwave_precision',
 'microwave_recall',
 'microwave_f1-score',
 'microwave_support',
 'dish washer_precision',
 'dish washer_recall',
 'dish washer_f1-score',
 'dish washer_support',
 'fridge_precision',
 'fridge_recall',
 'fridge_f1-score',
 'fridge_support',
 'kettle_precision',
 'kettle_recall',
 'kettle_f1-score',
 'kettle_support',
 'washer dryer_precision',
 'washer dryer_recall',
 'washer dryer_f1-score',
 'washer dryer_support',
 'toaster_precision',
 'toaster_recall',
 'toaster_f1-score',
 'toaster_support',
 'television_precision',
 'television_recall',
 'television_f1-score',
 'television_support',
 'micro avg_precision',
 'micro avg_recall',
 'micro avg_f1-score',
 'micro avg_support',
 'macro avg_precision',
 'macro avg_recall',
 'macro avg_f1-score',
 'macro avg_support',
 'weighted avg_precision',
 'weighted avg_recall',
 'weighted avg_f1-score',
 'weighted avg_support',
 'samples avg_precision',
 'samples avg_recall',
 'samples avg_f1-score',
 'samples avg_support'

In [62]:
complete_results.to_csv("complete_results.csv")